# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233540803505                   -0.50    8.0         
  2   -7.249838020777       -1.79       -1.39    1.0   9.49ms
  3   -7.251094410082       -2.90       -1.93    2.0   11.4ms
  4   -7.250961729016   +   -3.88       -1.87    2.0   11.8ms
  5   -7.251330152567       -3.43       -2.68    1.0   9.50ms
  6   -7.251337864953       -5.11       -3.09    1.0   9.87ms
  7   -7.251338692256       -6.08       -3.68    1.0   10.0ms
  8   -7.251338792797       -7.00       -4.12    2.0   11.9ms
  9   -7.251338798163       -8.27       -4.72    2.0   11.5ms
 10   -7.251338798636       -9.33       -5.16    2.0   11.7ms
 11   -7.251338798693      -10.24       -5.52    2.0   11.6ms
 12   -7.251338798703      -10.99       -6.16    1.0   9.84ms
 13   -7.251338798704      -12.01       -6.41    2.0   11.8ms
 14   -7.251338798705      -12.73       -7.00    1.0   10.1ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08046625504709679
[ Info: Arnoldi iteration step 2: normres = 0.5886504190342801
[ Info: Arnoldi iteration step 3: normres = 0.6513359002642987
[ Info: Arnoldi iteration step 4: normres = 0.1803372677800273
[ Info: Arnoldi iteration step 5: normres = 0.2732885847104777
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.09e-03, 2.03e-02, 2.17e-01, 1.60e-01, 3.59e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5798761032341905
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.21e-03, 4.38e-02, 5.55e-01, 1.36e-01, 7.05e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08619013126356262
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.25e-04, 4.65e-02, 5.80e-03, 4.48e-02, 4.73e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10719355462090249
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.73e-0